In [1]:
import numpy as np

import mne
mne.set_log_level(verbose='CRITICAL')
from mne.datasets import multimodal

import os
import glob
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

import mneflow
print(mneflow.__version__)

0.5.6


In [2]:
root = '/scratch/alr664/multiple_affix'
meg = root + '/meg'
logs = root + '/logs'

full_dataset = ["A0394", "A0421", "A0446", "A0451", "A0468", "A0484", "A0495", "A0502", "A0503", "A0508", 
                "A0509", "A0512", "A0513", "A0514", "A0516", "A0517", "A0518", "A0519", "A0520", "A0521", 
                "A0522", "A0523", "A0524", "A0525"]

In [3]:
subjects = [subj for subj in os.listdir(meg) if not subj.startswith('.')]
subjects
len(subjects)

24

In [4]:
epochs_list = []

for subject in full_dataset:
    subj_epoch_path = meg + '/' + subject + '/' + subject + '_rejection-epo.fif'
    print(subj_epoch_path)
    subj_epoch = mne.read_epochs(subj_epoch_path)
    subj_epoch = subj_epoch.resample(125)
    subj_epoch_mag = subj_epoch.pick_types(meg='mag')
    epochs_list.append(subj_epoch_mag)

print("Done!")

epochs = mne.concatenate_epochs(epochs_list, on_mismatch='ignore')

print("Done!.")

/scratch/alr664/multiple_affix/meg/A0394/A0394_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0421/A0421_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0446/A0446_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0451/A0451_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0468/A0468_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0484/A0484_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0495/A0495_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0502/A0502_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0503/A0503_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0508/A0508_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0509/A0509_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0512/A0512_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0513/A0513_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0514/A0514_rejection-epo.fif
/scratch/alr664/multiple_affix/meg/A0516/A0516_rejection-epo.fif
/scratch/alr664/multiple_

In [5]:
epochs.get_data().shape

(45264, 207, 100)

In [6]:
epochs.event_id

{'0Suff w/ Lat': 1,
 '0Suff w/o Lat.': 2,
 '0Suff NW': 4,
 '1Suff w/ Lat.': 11,
 '1Suff w/o Lat.': 12,
 '1Suff PseudoStemNW': 14,
 '1Suff RealStemNW': 15,
 '2Suff w/ Lat.': 21,
 '2Suff w/o Lat.': 22,
 '2Suff Composite': 23,
 '2Suff PseudoStemNW': 24,
 '2Suff RealStemNW': 25}

In [7]:
from collections import Counter
Counter([event[2] for event in epochs.events])

Counter({22: 7200,
         4: 6168,
         12: 6072,
         2: 5448,
         24: 4992,
         25: 4656,
         14: 3648,
         15: 3168,
         23: 1536,
         21: 912,
         11: 744,
         1: 720})

In [8]:
epochs.times

array([-0.2  , -0.192, -0.184, -0.176, -0.168, -0.16 , -0.152, -0.144,
       -0.136, -0.128, -0.12 , -0.112, -0.104, -0.096, -0.088, -0.08 ,
       -0.072, -0.064, -0.056, -0.048, -0.04 , -0.032, -0.024, -0.016,
       -0.008,  0.   ,  0.008,  0.016,  0.024,  0.032,  0.04 ,  0.048,
        0.056,  0.064,  0.072,  0.08 ,  0.088,  0.096,  0.104,  0.112,
        0.12 ,  0.128,  0.136,  0.144,  0.152,  0.16 ,  0.168,  0.176,
        0.184,  0.192,  0.2  ,  0.208,  0.216,  0.224,  0.232,  0.24 ,
        0.248,  0.256,  0.264,  0.272,  0.28 ,  0.288,  0.296,  0.304,
        0.312,  0.32 ,  0.328,  0.336,  0.344,  0.352,  0.36 ,  0.368,
        0.376,  0.384,  0.392,  0.4  ,  0.408,  0.416,  0.424,  0.432,
        0.44 ,  0.448,  0.456,  0.464,  0.472,  0.48 ,  0.488,  0.496,
        0.504,  0.512,  0.52 ,  0.528,  0.536,  0.544,  0.552,  0.56 ,
        0.568,  0.576,  0.584,  0.592])

In [9]:
epochs = epochs.crop(tmin=0. , tmax= 0.6)

In [10]:
epochs.times

array([0.   , 0.008, 0.016, 0.024, 0.032, 0.04 , 0.048, 0.056, 0.064,
       0.072, 0.08 , 0.088, 0.096, 0.104, 0.112, 0.12 , 0.128, 0.136,
       0.144, 0.152, 0.16 , 0.168, 0.176, 0.184, 0.192, 0.2  , 0.208,
       0.216, 0.224, 0.232, 0.24 , 0.248, 0.256, 0.264, 0.272, 0.28 ,
       0.288, 0.296, 0.304, 0.312, 0.32 , 0.328, 0.336, 0.344, 0.352,
       0.36 , 0.368, 0.376, 0.384, 0.392, 0.4  , 0.408, 0.416, 0.424,
       0.432, 0.44 , 0.448, 0.456, 0.464, 0.472, 0.48 , 0.488, 0.496,
       0.504, 0.512, 0.52 , 0.528, 0.536, 0.544, 0.552, 0.56 , 0.568,
       0.576, 0.584, 0.592])

In [11]:
epochs.get_data().shape

(45264, 207, 75)

In [12]:
path = './data/'
data_id = 'meg_epochs_12'

import_opt = dict(path=path,
                  data_id=data_id,
                  input_type='trials',
                  target_type='int',
                  n_folds= 5,
                  test_set = 'holdout',
                  overwrite=True,
                  picks={'meg':'grad'},
                  scale=False,
                  crop_baseline=False,
                  decimate=None,
                  )

In [13]:
meta = mneflow.produce_tfrecords(epochs, **import_opt)

processing epochs
Input shapes: X (n, ch, t) :  (45264, 207, 75) y (n, [signal_channels], y_shape) :  (45264, 1) 
 input_type :  trials target_type :  int segment_y :  False
Preprocessing:
n: 45264
Splitting into: 6 folds x 7544
Preprocessed: (45264, 1, 75, 207) (45264, 1) folds: 6 x 7544
Preprocessed targets:  (45264, 1)
Prepocessed sample shape: (1, 75, 207)
Target shape actual/metadata:  (12,) (12,)
Saving TFRecord# 0
Updating: meta.data
Updating: meta.preprocessing


In [14]:
dataset = mneflow.Dataset(meta, train_batch=64)

2024-06-05 22:22:25.813966: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


Updating: meta.data


In [15]:
lfcnn_params = dict(n_latent=32,
                  filter_length=7,
                  nonlin = tf.nn.relu,
                  padding = 'SAME',
                  pooling = 2,
                  pool_type='max',
                  dropout = .5,
                  l1_scope = ["weights"],
                  l1=3e-4)

meta.update(model_specs=lfcnn_params)

model = mneflow.models.LFCNN(meta)
model.build()

Updating: meta.model_specs
Updating: meta.data
Setting reg for dmx, to l1
Built: dmx input: (None, 1, 75, 207)
Setting reg for tconv, to l1
Built: tconv input: (None, 1, 75, 32)
Setting reg for fc, to l1
Built: fc input: (None, 1, 38, 32)
Input shape: (1, 75, 207)
y_pred: (None, 12)
Initialization complete!


In [16]:
model.train(n_epochs=20, eval_step=50, mode='cv')

Updating: meta.train_params
Class weights:  None
Running cross-validation with 5 folds
fold: 0
Epoch 1/20


I0000 00:00:1717640552.019384  567831 service.cc:145] XLA service 0x14a6a4008330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1717640552.019444  567831 service.cc:153]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2024-06-05 22:22:33.181535: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-05 22:22:34.995408: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
I0000 00:00:1717640563.565055  567831 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 - 18s - 366ms/step - cat_ACC: 0.1366 - loss: 2.6524 - val_cat_ACC: 0.1583 - val_loss: 2.5744
Epoch 2/20
50/50 - 1s - 21ms/step - cat_ACC: 0.1497 - loss: 2.5699 - val_cat_ACC: 0.1583 - val_loss: 2.5468
Epoch 3/20
50/50 - 2s - 35ms/step - cat_ACC: 0.1525 - loss: 2.5319 - val_cat_ACC: 0.1583 - val_loss: 2.5144
Epoch 4/20
50/50 - 2s - 32ms/step - cat_ACC: 0.1388 - loss: 2.5220 - val_cat_ACC: 0.1583 - val_loss: 2.4900
Epoch 5/20
50/50 - 2s - 36ms/step - cat_ACC: 0.1491 - loss: 2.5026 - val_cat_ACC: 0.1583 - val_loss: 2.4682
Epoch 6/20
50/50 - 2s - 32ms/step - cat_ACC: 0.1491 - loss: 2.4787 - val_cat_ACC: 0.1583 - val_loss: 2.4472
Epoch 7/20
50/50 - 2s - 30ms/step - cat_ACC: 0.1475 - loss: 2.4519 - val_cat_ACC: 0.1583 - val_loss: 2.4299
Epoch 8/20
50/50 - 2s - 37ms/step - cat_ACC: 0.1465 - loss: 2.4415 - val_cat_ACC: 0.1583 - val_loss: 2.4192
Epoch 9/20
50/50 - 1s - 23ms/step - cat_ACC: 0.1525 - loss: 2.4186 - val_cat_ACC: 0.1583 - val_loss: 2.4038
Epoch 10/20
50/50 - 2s - 30ms/step - 

2024-06-05 22:23:44.458896: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Re-shuffling weights between folds
Fold: 0 Validation performance:

                      Loss: 2.3566, 
                      Metric: 0.1583
fold: 1
Epoch 1/20
50/50 - 3s - 58ms/step - cat_ACC: 0.1459 - loss: 2.4188 - val_cat_ACC: 0.1558 - val_loss: 2.3784
Epoch 2/20
50/50 - 1s - 20ms/step - cat_ACC: 0.1678 - loss: 2.3782 - val_cat_ACC: 0.1558 - val_loss: 2.3749
Epoch 3/20
50/50 - 2s - 31ms/step - cat_ACC: 0.1581 - loss: 2.3548 - val_cat_ACC: 0.1558 - val_loss: 2.3698
Test performance:
                          Loss: 2.3821 
                          Metric: 0.1560


...

196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Re-shuffling weights between folds
Fold: 2 Validation performance:

                      Loss: 2.3700, 
                      Metric: 0.1641
fold: 3
Epoch 1/20
50/50 - 3s - 55ms/step - cat_ACC: 0.1450 - loss: 2.4298 - val_cat_ACC: 0.1540 - val_loss: 2.3773
Epoch 2/20
50/50 - 1s - 22ms/step - cat_ACC: 0.1669 - loss: 2.3682 - val_cat_ACC: 0.1540 - val_loss: 2.3719
Epoch 3/20
50/50 - 2s - 31ms/step - cat_ACC: 0.1587 - loss: 2.3656 - val_cat_ACC: 0.1540 - val_loss: 2.3654
Test performance:
                          Loss: 2.3875 
                          Metric: 0.1560


2024-06-05 22:25:21.876848: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Re-shuffling weights between folds
Fold: 3 Validation performance:

                      Loss: 2.3773, 
                      Metric: 0.1540
fold: 4
Epoch 1/20
50/50 - 3s - 58ms/step - cat_ACC: 0.1384 - loss: 2.4219 - val_cat_ACC: 0.1646 - val_loss: 2.3712
Epoch 2/20
50/50 - 1s - 19ms/step - cat_ACC: 0.1591 - loss: 2.3917 - val_cat_ACC: 0.1646 - val_loss: 2.3674
Epoch 3/20
50/50 - 2s - 30ms/step - cat_ACC: 0.1644 - loss: 2.3623 - val_cat_ACC: 0.1646 - val_loss: 2.3603
Test performance:
                          Loss: 2.3897 
                          Metric: 0.1560


2024-06-05 22:25:54.407862: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


198/198 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold: 4 Validation performance:

                      Loss: 2.3712, 
                      Metric: 0.1646
cv with 5 folds completed. Loss: 2.3707 +/- 0.0078. Metric: 0.1594 +/- 0.0043
cv with 5 fold(s) completed. 

              Validation Performance: 
              Loss: 2.3707 +/- 0.0078.
              Metric: 0.1594 +/- 0.0043


              Test Performance: 
              Loss: 2.3831 +/- 0.0059.
              Metric: 0.1560 +/- 0.0000
Saving updated log to:  ./data/models\/lfcnn_log.csv


In [17]:
test_loss, test_acc = model.evaluate(meta.data['test_paths'])
print("Test set: Loss = {:.4f} Accuracy = {:.4f}".format(test_loss, test_acc))

Test set: Loss = 2.3897 Accuracy = 0.1560


In [18]:
model = mneflow.models.VARCNN(meta)
model.build()

Updating: meta.data
Setting reg for dmx, to l1
Built: dmx input: (None, 1, 75, 207)
input_shape: (None, 1, 75, 32)
Setting reg for tconv, to l1
Built: tconv input: (None, 1, 75, 32)
Setting reg for fc, to l1
Built: fc input: (None, 1, 38, 32)
Input shape: (1, 75, 207)
y_pred: (None, 12)
Initialization complete!


In [19]:
model.train(n_epochs=20, eval_step=50, mode='cv')

Updating: meta.train_params
Class weights:  None
Running cross-validation with 5 folds
fold: 0
Epoch 1/20
50/50 - 8s - 158ms/step - cat_ACC: 0.1400 - loss: 2.8188 - val_cat_ACC: 0.1297 - val_loss: 2.7502
Epoch 2/20
50/50 - 1s - 23ms/step - cat_ACC: 0.1360 - loss: 2.7590 - val_cat_ACC: 0.1642 - val_loss: 2.7113
Epoch 3/20
50/50 - 1s - 28ms/step - cat_ACC: 0.1440 - loss: 2.6931 - val_cat_ACC: 0.1642 - val_loss: 2.6715
Epoch 4/20
50/50 - 1s - 28ms/step - cat_ACC: 0.1380 - loss: 2.6562 - val_cat_ACC: 0.1372 - val_loss: 2.6422
Epoch 5/20
50/50 - 2s - 32ms/step - cat_ACC: 0.1572 - loss: 2.6251 - val_cat_ACC: 0.1642 - val_loss: 2.6142
Epoch 6/20
50/50 - 2s - 33ms/step - cat_ACC: 0.1584 - loss: 2.5834 - val_cat_ACC: 0.1642 - val_loss: 2.5913
Epoch 7/20
50/50 - 1s - 28ms/step - cat_ACC: 0.1580 - loss: 2.5838 - val_cat_ACC: 0.1642 - val_loss: 2.5736
Epoch 8/20
50/50 - 1s - 28ms/step - cat_ACC: 0.1564 - loss: 2.5728 - val_cat_ACC: 0.1642 - val_loss: 2.5542
Epoch 9/20
50/50 - 1s - 29ms/step - cat_

2024-06-04 19:11:29.186159: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


199/199 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Re-shuffling weights between folds
Fold: 0 Validation performance:

                      Loss: 2.4715, 
                      Metric: 0.1642
fold: 1
Epoch 1/20
50/50 - 3s - 50ms/step - cat_ACC: 0.1580 - loss: 2.5006 - val_cat_ACC: 0.1543 - val_loss: 2.4751
Epoch 2/20
50/50 - 1s - 24ms/step - cat_ACC: 0.1448 - loss: 2.4893 - val_cat_ACC: 0.1543 - val_loss: 2.4631
Epoch 3/20
50/50 - 2s - 32ms/step - cat_ACC: 0.1512 - loss: 2.4570 - val_cat_ACC: 0.1543 - val_loss: 2.4548
Epoch 4/20
50/50 - 1s - 29ms/step - cat_ACC: 0.1452 - loss: 2.4499 - val_cat_ACC: 0.1543 - val_loss: 2.4487
Epoch 5/20
50/50 - 1s - 28ms/step - cat_ACC: 0.1516 - loss: 2.4534 - val_cat_ACC: 0.1543 - val_loss: 2.4425
Epoch 6/20
50/50 - 1s - 29ms/step - cat_ACC: 0.1656 - loss: 2.4219 - val_cat_ACC: 0.1543 - val_loss: 2.4378
Epoch 7/20
50/50 - 1s - 28ms/step - cat_ACC: 0.1540 - loss: 2.4320 - val_cat_ACC: 0.1543 - val_loss: 2.4327
Epoch 8/20
50/50 - 1s - 29ms/step - cat_ACC: 0.1672 -

2024-06-04 19:12:24.298684: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Re-shuffling weights between folds
Fold: 1 Validation performance:

                      Loss: 2.3791, 
                      Metric: 0.1543
fold: 2
Epoch 1/20
50/50 - 2s - 47ms/step - cat_ACC: 0.1400 - loss: 2.4250 - val_cat_ACC: 0.1387 - val_loss: 2.3961
Epoch 2/20
50/50 - 1s - 24ms/step - cat_ACC: 0.1476 - loss: 2.4100 - val_cat_ACC: 0.1611 - val_loss: 2.3757
Epoch 3/20
50/50 - 2s - 34ms/step - cat_ACC: 0.1656 - loss: 2.3664 - val_cat_ACC: 0.1611 - val_loss: 2.3692
Epoch 4/20
50/50 - 1s - 29ms/step - cat_ACC: 0.1504 - loss: 2.3730 - val_cat_ACC: 0.1320 - val_loss: 2.3677
Epoch 5/20
50/50 - 1s - 29ms/step - cat_ACC: 0.1508 - loss: 2.3793 - val_cat_ACC: 0.1611 - val_loss: 2.3601
Epoch 6/20
50/50 - 1s - 28ms/step - cat_ACC: 0.1668 - loss: 2.3470 - val_cat_ACC: 0.1611 - val_loss: 2.3565
Epoch 7/20
50/50 - 1s - 29ms/step - cat_ACC: 0.1656 - loss: 2.3775 - val_cat_ACC: 0.1611 - val_loss: 2.3562
Epoch 8/20
50/50 - 1s - 28ms/step - cat_ACC: 0.1688 -

...

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Re-shuffling weights between folds
Fold: 2 Validation performance:

                      Loss: 2.3222, 
                      Metric: 0.1320
fold: 3
Epoch 1/20
50/50 - 2s - 46ms/step - cat_ACC: 0.1476 - loss: 2.3783 - val_cat_ACC: 0.1597 - val_loss: 2.3460
Epoch 2/20
50/50 - 1s - 19ms/step - cat_ACC: 0.1464 - loss: 2.3543 - val_cat_ACC: 0.1597 - val_loss: 2.3375
Epoch 3/20
50/50 - 1s - 29ms/step - cat_ACC: 0.1504 - loss: 2.3313 - val_cat_ACC: 0.1597 - val_loss: 2.3384
Test performance:
                          Loss: 2.3529 
                          Metric: 0.1629


2024-06-04 19:13:48.657297: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Re-shuffling weights between folds
Fold: 3 Validation performance:

                      Loss: 2.3460, 
                      Metric: 0.1597
fold: 4
Epoch 1/20
50/50 - 2s - 46ms/step - cat_ACC: 0.1516 - loss: 2.3851 - val_cat_ACC: 0.1345 - val_loss: 2.3511
Epoch 2/20
50/50 - 1s - 20ms/step - cat_ACC: 0.1440 - loss: 2.3670 - val_cat_ACC: 0.1571 - val_loss: 2.3436
Epoch 3/20
50/50 - 2s - 30ms/step - cat_ACC: 0.1508 - loss: 2.3448 - val_cat_ACC: 0.1344 - val_loss: 2.3366
Test performance:
                          Loss: 2.3604 
                          Metric: 0.1340


2024-06-04 19:14:18.637518: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Fold: 4 Validation performance:

                      Loss: 2.3511, 
                      Metric: 0.1345
cv with 5 folds completed. Loss: 2.3740 +/- 0.0520. Metric: 0.1490 +/- 0.0132
cv with 5 fold(s) completed. 

              Validation Performance: 
              Loss: 2.3740 +/- 0.0520.
              Metric: 0.1490 +/- 0.0132


              Test Performance: 
              Loss: 2.3813 +/- 0.0484.
              Metric: 0.1505 +/- 0.0152
Saving updated log to:  ./data/models\/varcnn_log.csv


In [18]:
test_loss, test_acc = model.evaluate(meta.data['test_paths'])
print("Test set: Loss = {:.4f} Accuracy = {:.4f}".format(test_loss, test_acc))

Test set: Loss = 2.3897 Accuracy = 0.1560
